<br><br><br><br><br>

# Fun exercise with Numpy

<br><br><br><br><br>

In [62]:
def png(rgb):    # Numpy is concise enough to create PNG files from raw bytes in one screenful of code
    def chunk(tag, data):
        out = numpy.empty(4 + len(tag) + len(data) + 4, "u1")   # each chunk consists of:
        out[0:4] = numpy.array([len(data)], ">u4").view("u1")   #   4-byte data size
        out[4:8] = numpy.frombuffer(tag, "u1")                  #   4-byte tag (data type)
        out[8:8 + len(data)] = numpy.frombuffer(data, "u1")     #   the data
        crc = zlib.crc32(data, zlib.crc32(tag))                 #   CRC: cyclic redundancy check
        out[-4:] = numpy.array([crc & 0xffffffff], ">u4").view("u1")
        return out

    preamble = numpy.frombuffer(b"\x89PNG\r\n\x1a\n", "u1")     # preamble: "this is a PNG file"
    width_height = numpy.array(rgb.shape[1::-1], ">u4")         # header: size + image type (RGBA)
    header = numpy.concatenate([width_height.view("u1"), numpy.array([8, 6, 0, 0, 0], "u1")])
    headerchunk = chunk(b"IHDR", header.tostring())

    mini, maxi = rgb.min(), rgb.max()                           # normalize the value ranges
    rgb = (255 / (maxi - mini))*(rgb - mini).astype("u1")       # and convert to one byte per channel
    data = numpy.empty((rgb.shape[0], 1 + 4*rgb.shape[1]), "u1")
    data[:, 0], data[:, 4::4] = 0, 255                          # beginning of scanline and opacity
    data[:, 1::4], data[:, 2::4], data[:, 3::4] = rgb[:, :, 0], rgb[:, :, 1], rgb[:, :, 2]  # R, G, B
    datachunk = chunk(b"IDAT", zlib.compress(data.tostring()))  # scanlines are zlib-compressed

    endchunk = chunk(b"IEND", b"")                              # IEND means no more chunks
    return numpy.concatenate([preamble, headerchunk, datachunk, endchunk])

In [63]:
import numpy
import zlib
import IPython.display

def show(rgb):
    return IPython.display.display(IPython.display.Image(data=png(rgb)))

rgb = numpy.zeros((10, 20, 3))
rgb[:5, :10, 0] = 1
show(rgb)